# 1.2.1 List of stocks



## Data source:

ข้อมูลบริษัท/หลักทรัพย์ -> ตราสารทุน -> รายชื่อบริษัท / หลักทรัพย์

https://www.set.or.th/th/company/companylist.html

รายละเอียด รายชื่อบริษัทจดทะเบียนในตลาดหลักทรัพย์ (ปรับปรุงข้อมูลทุกวันทำการแรกของแต่ละสัปดาห์) https://www.set.or.th/dat/eod/listedcompany/static/listedCompanies_th_TH.xls

The Excel file **securities.xls** contains the list of all ticker symbols. It was downloaded from the Stock Exchange of Thailand website. The output of this process is CSV file **stock_ticker.csv**.

## Process

1. Download data from URL
2. Decode data from TIS-620
3. Parse HTML content to XML tree
4. Use xpath to extract each row and transform
5. Filter only SET
6. Show data to inspect data frame
7. Output saved to 'stock_ticker.csv'


### Required Python Modules

~~~
requests
pandas
lxml
~~~

In [24]:
# Import required library

import pandas as pd
import requests
from lxml import etree

In [25]:
# Download data

src = "https://www.set.or.th/dat/eod/listedcompany/static/listedCompanies_th_TH.xls"
r = requests.get(src)
content = r.content.decode('tis-620')

r.status_code, content[:300] # sample first 300 characters

(200,
 '<table cellSpacing="0" cellPadding="0" width="100%" border="0">\n  <tr>\n    <td colspan="6" align="left"><strong>รายชื่อบริษัทจดทะเบียนในตลาดหลักทรัพย์แห่งประเทศไทย</strong></td>\n    <td colspan="4" align="right"><strong>ข้อมูล ณ วันที่ 07 เม.ย. 2563<strong></td>\n  </tr>\n  <tr align="center" bgcolor=')

In [26]:
# Decode and parse

html = etree.HTML(r.content.decode('tis-620'))

In [27]:
# Extract rows

stocks = {}

class Company:
    def __init__(self, row):
        self.exchange = row.getchildren()[2].text.strip().upper()
        self.symbol = row.getchildren()[0].text.strip().upper()
        self.name = row.getchildren()[1].text.strip()
        self.industry = row.getchildren()[3].text.strip()
        self.sector = row.getchildren()[4].text.strip()
        self.address = row.getchildren()[5].text.strip()
        self.zipcode = row.getchildren()[6].text.strip()
        self.telephone = row.getchildren()[7].text.strip()
        self.fax = row.getchildren()[8].text.strip()
        self.zipcode = row.getchildren()[9].text.strip()
        self._row = row
        
    def __str__(self):
        return """<Company %s:%s >""" % (self.exchange, self.symbol,)

for row in html.xpath('//tr'):
    print(len(row.getchildren()))
    if (len(row.getchildren()) == 10) and (row.getchildren()[0].text is not None):
        for c in row.getchildren():
            print(c.text)
        try:
            company = Company(row)
            print(company)
            stocks[company.symbol] = company
        except Exception as e:
            print("Error for %s %s" % (str(e), row))

2
10
10
2S                  
บริษัท 2 เอส เมทัล จำกัด (มหาชน)
mai     
สินค้าอุตสาหกรรม 
-
8/5 หมู่ที่ 14 ตำบลท่าช้าง อำเภอบางกล่ำ สงขลา
90110
0-7480-0111
0-7480-1286
www.ss.co.th
<Company MAI:2S >
10
7UP                 
บริษัท เซเว่น ยูทิลิตี้ส์ แอนด์ พาวเวอร์ จำกัด (มหาชน)
SET     
ทรัพยากร
พลังงานและสาธารณูปโภค
73 อาคารมหาชล ซอยสุขุมวิท 62  ถนนสุขุมวิท แขวงพระโขนงใต้ เขตพระโขนง กทม.
10260
0-2741-5700
0-2741-5724
www.sevenup.co.th
<Company SET:7UP >
10
A                   
บริษัท อารียา พรอพเพอร์ตี้ จำกัด (มหาชน)
SET     
อสังหาริมทรัพย์และก่อสร้าง
พัฒนาอสังหาริมทรัพย์
999 ถนนประดิษฐ์มนูธรรม แขวงสะพานสอง เขตวังทองหลาง กทม.
10310
0-2798-9999, 1797
0-2798-9990
http://www.areeya.co.th
<Company SET:A >
10
A5                  
บริษัท แอสเซท ไฟว์ กรุ๊ป จำกัด (มหาชน)
mai     
 บริการ
-
เลขที่ 349 อาคารเอสเจ อินฟินิท วัน บิสซิเนส คอมเพล็กซ์ ชั้น ที่ 19 ยูนิต 1901 ถนนวิภาวดีรังสิต แขวงจอมพล เขตจตุจักร กทม.
10900
0-2026-3512
0-2026-3513
www.assetfive.co.th
<Company MAI:A5 >
10
AAV            

บริษัท กรุงเทพดุสิตเวชการ จำกัด(มหาชน)
SET     
บริการ
การแพทย์
เลขที่ 2 ซอยศูนย์วิจัย 7 ถนนเพชรบุรีตัดใหม่ แขวงบางกะปิ เขตห้วยขวาง กทม.
10310
0-2310-3000
0-2318-1546 , 0-2310-3327
www.bangkokhospital.com
<Company SET:BDMS >
10
BEAUTY              
บริษัท บิวตี้ คอมมูนิตี้ จำกัด (มหาชน)
SET     
บริการ
พาณิชย์
เลขที่ 10/915 ซอยนวลจันทร์ 34 แขวงนวลจันทร์ เขตบึงกุ่ม กทม.
10230
0-2946-0700-6
0-2946-1571
WWW.BEAUTYCOMMUNITY.CO.TH
<Company SET:BEAUTY >
10
BEC                 
บริษัท บีอีซี เวิลด์ จำกัด (มหาชน)
SET     
บริการ
สื่อและสิ่งพิมพ์
อาคารมาลีนนท์ ทาวเวอร์, 3199 ถนนพระรามสี่ แขวงคลองตัน เขตคลองเตย กทม.
10110
0-2204-3333, 0-2262-3333
0-2204-1384
www.becworld.com
<Company SET:BEC >
10
BEM                 
บริษัท ทางด่วนและรถไฟฟ้ากรุงเทพ จำกัด (มหาชน)
SET     
บริการ
ขนส่งและโลจิสติกส์
587 ถนนสุทธิสารวินิจฉัย แขวงรัชดาภิเษก เขตดินแดง กทม.
10400
0-2641-4611, 0-2354-2000
0-2641-4610, 0-2354-2040
www.bemplc.co.th
<Company SET:BEM >
10
BFIT                
บริษัทเงินทุน ศรีสวัสดิ์ จำกัด (

เลขที่ 999/9 ถนนพระราม 1 แขวงปทุมวัน เขตปทุมวัน กทม.
10330
0-2667-5555 ต่อ 1660
0-2667-5590
www.cpnreit.com
<Company SET:CPNREIT >
10
CPR                 
บริษัท ซีพีอาร์ โกมุ อินดัสเตรียล จำกัด (มหาชน)
mai     
สินค้าอุตสาหกรรม 
-
78 หมู่ 2 ถนนพหลโยธิน (กิโลเมตรที่ 77.5) ตำบลสนับทึบ อำเภอวังน้อย พระนครศรีอยุธยา
13170
0-3572-3324-7, 0-3572-3128-31
0-3572-3323, 0-3572-3332
www.cprgomu.co.th
<Company MAI:CPR >
10
CPT                 
บริษัท ซีพีที ไดร์ แอนด์ เพาเวอร์ จำกัด (มหาชน)
SET     
สินค้าอุตสาหกรรม
วัสดุอุตสาหกรรมและเครื่องจักร
เลขที่ 230/7 ถนนเทศบาลรังรักษ์เหนือ แขวงลาดยาว เขตจตุจักร กทม.
10900
0-2954-2590-2
0-2580-6332-4
www.CPTthailand.com
<Company SET:CPT >
10
CPTGF               
กองทุนรวมสิทธิการเช่าอสังหาริมทรัพย์ ซี.พี.ทาวเวอร์ โกรท
SET     
อสังหาริมทรัพย์และก่อสร้าง
กองทุนรวมอสังหาริมทรัพย์และกองทรัสต์เพื่อการลงทุนในอสังหาริมทรัพย์
เลขที่ 1 อาคารเอ็มไพร์ทาวเวอร์ ชั้น 32 ถนนสาทรใต้  แขวงยานนาวา เขตสาทร กทม.
10120
0-2686-6100
0-2670-0430
www.ktam.co.th
<Company SET:CPTGF 

อสังหาริมทรัพย์และก่อสร้าง
วัสดุก่อสร้าง
44/2 หมู่ 2 ถนนติวานนท์ ตำบลบางกระดี อำเภอเมือง ปทุมธานี
12000
0-2501-1055, 0-2501-2462-7, 0-2501-2020
0-2501-2134, 0-2501-2468
www.gel.co.th
<Company SET:GEL >
10
GENCO               
บริษัทบริหารและพัฒนาเพื่อการอนุรักษ์สิ่งแวดล้อม จำกัด(มหาชน)
SET     
บริการ
บริการเฉพาะกิจ
447 ถนนบอนด์สตรีท ตำบลบางพูด อำเภอปากเกร็ด นนทบุรี
11120
0-2502-0900
0-2502-0999
http://www.genco.co.th
<Company SET:GENCO >
10
GFPT                
บริษัท จีเอฟพีที จำกัด (มหาชน)
SET     
เกษตรและอุตสาหกรรมอาหาร
ธุรกิจการเกษตร
312 ถนนพระราม 2 แขวงบางมด เขตจอมทอง กทม.
10150
0-2473-8000
0-2473-8392, 0-2473-8384
www.gfpt.co.th
<Company SET:GFPT >
10
GGC                 
บริษัท โกลบอลกรีนเคมิคอล จำกัด (มหาชน)
SET     
สินค้าอุตสาหกรรม
ปิโตรเคมีและเคมีภัณฑ์
เลขที่ 555/1 ศูนย์เอนเนอร์ยี่คอมเพล็กซ์ อาคารเอ ชั้น 4 ถนนวิภาวดีรังสิต แขวงจตุจักร เขตจตุจักร กทม.
10900
0-2558-7300
0-2558-7301
http://www.ggcplc.com
<Company SET:GGC >
10
GIFT                
บริษัท แกรททิทูด อินฟินิท จำก

LH                  
บริษัทแลนด์แอนด์เฮ้าส์ จำกัด (มหาชน)
SET     
อสังหาริมทรัพย์และก่อสร้าง
พัฒนาอสังหาริมทรัพย์
อาคารคิวเฮ้าส์ ลุมพินี ชั้น 37-38, 1 ถนนสาทรใต้ แขวงทุ่งมหาเมฆ เขตสาทร กทม.
10120
0-2343-8900
0-2230-8130
www.lh.co.th
<Company SET:LH >
10
LHFG                
บริษัท แอล เอช ไฟแนนซ์เชียล กรุ๊ป จำกัด (มหาชน)
SET     
ธุรกิจการเงิน
ธนาคาร
เลขที่ 1 ชั้น 5 อาคารคิวเฮ้าส์ ลุมพินี ถนนสาทรใต้ แขวงทุ่งมหาเมฆ เขตสาทร กทม.
10120
0-2359-0000
0-2677-7223
https://www.lhfg.co.th
<Company SET:LHFG >
10
LHHOTEL             
ทรัสต์เพื่อการลงทุนในสิทธิการเช่าอสังหาริมทรัพย์ แอล เอช โฮเทล
SET     
อสังหาริมทรัพย์และก่อสร้าง
กองทุนรวมอสังหาริมทรัพย์และกองทรัสต์เพื่อการลงทุนในอสังหาริมทรัพย์
เลขที่ 11 อาคารคิวเฮ้าส์ สาทร ชั้น 14 ถนนสาทรใต้ แขวงทุ่งมหาเมฆ เขตสาทร กทม.
10120
0-2286-3484
0-2286-3585
www.lhfund.co.th
<Company SET:LHHOTEL >
10
LHK                 
บริษัท โลหะกิจ เม็ททอล จำกัด (มหาชน)
SET     
สินค้าอุตสาหกรรม
เหล็ก
เลขที่ 66/1 หมู่ที่ 6 ซอยสุขสวัสดิ์ 76 ถนนสุขสวัสดิ์ ตำบลบางจาก อ

บริการ
สื่อและสิ่งพิมพ์
เลขที่ 1858/121-122, 1858/125-128 ชั้นที่ 28,30,31 ถนนเทพรัตน แขวงบางนาใต้ เขตบางนา กทม.
10260
None
None
http://www.nationgroup.com
Error for 'NoneType' object has no attribute 'strip' <Element tr at 0x7fb41c3d8a00>
10
NNCL                
บริษัท นวนคร จำกัด (มหาชน)
SET     
อสังหาริมทรัพย์และก่อสร้าง
พัฒนาอสังหาริมทรัพย์
เลขที่ 999 หมู่ 13 ถนนพหลโยธิน ต.คลองหนึ่ง อ.คลองหลวง ปทุมธานี
12120
0-2529-0031-5
0-2529-2176
www.navanakorn.co.th
<Company SET:NNCL >
10
NOBLE               
บริษัท โนเบิล ดีเวลลอปเมนท์ จำกัด (มหาชน)
SET     
อสังหาริมทรัพย์และก่อสร้าง
พัฒนาอสังหาริมทรัพย์
อาคารโนเบิล เลขที่ 1035 ถนนเพลินจิต แขวงลุมพินี เขตปทุมวัน กทม.
10330
0-2251-9955
0-2251-9977
www.noblehome.com
<Company SET:NOBLE >
10
NOK                 
บริษัท สายการบินนกแอร์ จำกัด (มหาชน)
SET     
บริการ
ขนส่งและโลจิสติกส์
183 ชั้น 17 อาคารรัจนาการ ถนนสาทรใต้ แขวงยานนาวา เขตสาทร กทม.
10120
0-2627-2000
0-2286-9830
www.nokair.com
<Company SET:NOK >
10
NPK                 
บริษัท นิวพลัส

http://www.rajaferryport.com
<Company MAI:RP >
10
RPC                 
บริษัท อาร์พีซีจี จำกัด (มหาชน)
SET     
ทรัพยากร
พลังงานและสาธารณูปโภค
เลขที่ 86/2 อาคารสัมมากรเพลส ถนนรามคำแหง  แขวงสะพานสูง เขตสะพานสูง กทม.
10240
0-2372-3600
0-2372-3327
www.rpcthai.com
<Company SET:RPC >
10
RPH                 
บริษัท โรงพยาบาลราชพฤกษ์ จำกัด (มหาชน)
SET     
บริการ
การแพทย์
บ้านเลขที่ 456 หมูที่ 14 ถนนมิตรภาพ ตำบลในเมือง อำเภอเมืองขอนแก่น ขอนแก่น
40000
0-4333-3555
None
http://www.rph.co.th
Error for 'NoneType' object has no attribute 'strip' <Element tr at 0x7fb41c4406e0>
10
RS                  
บริษัท อาร์เอส จำกัด (มหาชน)
SET     
บริการ
พาณิชย์
เลขที่ 27 อาคารอาร์เอสกรุ๊ป ทาวเวอร์ เอ ถนนประเสริฐมนูกิจ แขวงเสนานิคม เขตจตุจักร กทม.
10900
0-2037-8888
None
http://www.rs.co.th
Error for 'NoneType' object has no attribute 'strip' <Element tr at 0x7fb41c440280>
10
RSP                 
บริษัท ริช สปอร์ต จำกัด (มหาชน)
SET     
บริการ
พาณิชย์
116/20 ถนน ณ ระนอง แขวงคลองเตย เขตคลองเตย กทม.
10110
0-2249

<Company SET:SSTRT >
10
STA                 
บริษัท ศรีตรังแอโกรอินดัสทรี จำกัด (มหาชน)
SET     
เกษตรและอุตสาหกรรมอาหาร
ธุรกิจการเกษตร
10 ซอย 10 ถนนเพชรเกษม ตำบลหาดใหญ่ อำเภอหาดใหญ่ สงขลา
90110
0-7434-4663
0-7434-4676, 0-7434-4677, 0-7423-7423
www.sritranggroup.com
<Company SET:STA >
10
STANLY              
บริษัท ไทยสแตนเลย์การไฟฟ้า จำกัด (มหาชน)
SET     
สินค้าอุตสาหกรรม
ยานยนต์
29/3 หมู่ 1 ถนนบางพูน-รังสิต ตำบลบ้านกลาง อำเภอเมือง ปทุมธานี
12000
0-2581-5462
0-2581-5397
www.thaistanley.com
<Company SET:STANLY >
10
STAR                
บริษัท สตาร์ ยูนิเวอร์แซล เน็ตเวิร์ค จำกัด (มหาชน)
mai     
อสังหาริมทรัพย์และก่อสร้าง
-
เลขที่ 57 อาคารปาร์ค เวนเชอร์ อีโคเพล็กซ์ ห้องเลขที่ 1607A ชั้นที่ 16 ถนนวิทยุ แขวงลุมพินี เขตปทุมวัน กทม.
10330
0-2041-8162
0-2041-8162
www.staruniversalnetwork.com
<Company MAI:STAR >
10
STARK               
บริษัท สตาร์ค คอร์เปอเรชั่น จำกัด (มหาชน)
SET     
สินค้าอุตสาหกรรม
วัสดุอุตสาหกรรมและเครื่องจักร
เลขที่ 518/5 อาคารมณียา เซ็นเตอร์ ชั้นที่ 16 ถนนเพลินจิต แขว

ชั้น 19 อาคารเมืองไทย - ภัทร คอมเพล็กซ์ อาคาร เอ เลขที่ 252/25 ถนนรัชดาภิเษก แขวงห้วยขวาง เขตห้วยขวาง กทม.
10310
0-2305-9800
0-2305-9803-4
http://www.phatraasset.com
<Company SET:URBNPF >
10
UREKA               
บริษัท ยูเรกา ดีไซน์ จำกัด (มหาชน)
mai     
สินค้าอุตสาหกรรม 
-
เลขที่ 19 หมู่ที่ 11 ตำบลลาดสวาย  อำเภอลำลูกกา ปทุมธานี
12150
0-2192-3737
0-2192-3743-44
http://www.eurekadesign.co.th
<Company MAI:UREKA >
10
UT                  
บริษัท ยูเนี่ยนอุตสาหกรรมสิ่งทอ จำกัด (มหาชน)
SET     
สินค้าอุปโภคบริโภค
แฟชั่น
1828 ถนนสุขุมวิท แขวงพระโขนงใต้ เขตพระโขนง กทม.
10260
0-2311-5111-9
0-2332-3123
www.ut.co.th
<Company SET:UT >
10
UTP                 
บริษัท ยูไนเต็ด เปเปอร์ จำกัด (มหาชน)
SET     
สินค้าอุตสาหกรรม
กระดาษและวัสดุการพิมพ์
113 - 115 ถนนริมคลองประปา แขวงบางซื่อ เขตบางซื่อ กทม.
10800
0-2910-2700-8
0-2910-2709
www.unitedpaper.co.th
<Company SET:UTP >
10
UV                  
บริษัท ยูนิเวนเจอร์ จำกัด (มหาชน)
SET     
อสังหาริมทรัพย์และก่อสร้าง
พัฒนาอสังหาริมทรัพย์
57 ปาร์คเวนเชอร

In [28]:
"Total tickers", len(stocks)

('Total tickers', 731)

In [29]:
all_stocks = {}
for symbol, company in stocks.items():
  all_stocks[symbol] = {
        'symbol': company.symbol,
        'exchange': company.exchange,
        'industry': company.industry,
        'sector': company.sector,
        'name': company.name
  }  


In [34]:
all_symbols = pd.DataFrame(all_stocks)
stocks = all_symbols.T
stocks
SET_stocks = stocks[stocks['exchange']=='SET']
SET_stocks

,symbol,exchange,industry,sector,name
7UP,7UP,SET,ทรัพยากร,พลังงานและสาธารณูปโภค,บริษัท เซเว่น ยูทิลิตี้ส์ แอนด์ พาวเวอร์ จำกัด...
A,A,SET,อสังหาริมทรัพย์และก่อสร้าง,พัฒนาอสังหาริมทรัพย์,บริษัท อารียา พรอพเพอร์ตี้ จำกัด (มหาชน)
AAV,AAV,SET,บริการ,ขนส่งและโลจิสติกส์,บริษัท เอเชีย เอวิเอชั่น จำกัด (มหาชน)
ABPIF,ABPIF,SET,ทรัพยากร,พลังงานและสาธารณูปโภค,กองทุนรวมโครงสร้างพื้นฐานโรงไฟฟ้า อมตะ บี.กริม...
ACC,ACC,SET,สินค้าอุปโภคบริโภค,ของใช้ในครัวเรือนและสำนักงาน,บริษัท แอดวานซ์ คอนเนคชั่น คอร์ปอเรชั่น จำกัด ...
...,...,...,...,...,...
WP,WP,SET,ทรัพยากร,พลังงานและสาธารณูปโภค,บริษัท ดับบลิวพี เอ็นเนอร์ยี่ จำกัด (มหาชน)
WPH,WPH,SET,บริการ,การแพทย์,บริษัท โรงพยาบาลวัฒนแพทย์ ตรัง จำกัด (มหาชน)
YCI,YCI,SET,สินค้าอุตสาหกรรม,ปิโตรเคมีและเคมีภัณฑ์,บริษัท ยงไทย จำกัด (มหาชน)
ZEN,ZEN,SET,เกษตรและอุตสาหกรรมอาหาร,อาหารและเครื่องดื่ม,บริษัท เซ็น คอร์ปอเรชั่น กรุ๊ป จำกัด (มหาชน)


In [33]:
output_file = "stock_ticker.csv"

SET_stocks.to_csv(output_file)


SET_stocks

,symbol,exchange,industry,sector,name
7UP,7UP,SET,ทรัพยากร,พลังงานและสาธารณูปโภค,บริษัท เซเว่น ยูทิลิตี้ส์ แอนด์ พาวเวอร์ จำกัด...
A,A,SET,อสังหาริมทรัพย์และก่อสร้าง,พัฒนาอสังหาริมทรัพย์,บริษัท อารียา พรอพเพอร์ตี้ จำกัด (มหาชน)
AAV,AAV,SET,บริการ,ขนส่งและโลจิสติกส์,บริษัท เอเชีย เอวิเอชั่น จำกัด (มหาชน)
ABPIF,ABPIF,SET,ทรัพยากร,พลังงานและสาธารณูปโภค,กองทุนรวมโครงสร้างพื้นฐานโรงไฟฟ้า อมตะ บี.กริม...
ACC,ACC,SET,สินค้าอุปโภคบริโภค,ของใช้ในครัวเรือนและสำนักงาน,บริษัท แอดวานซ์ คอนเนคชั่น คอร์ปอเรชั่น จำกัด ...
...,...,...,...,...,...
WP,WP,SET,ทรัพยากร,พลังงานและสาธารณูปโภค,บริษัท ดับบลิวพี เอ็นเนอร์ยี่ จำกัด (มหาชน)
WPH,WPH,SET,บริการ,การแพทย์,บริษัท โรงพยาบาลวัฒนแพทย์ ตรัง จำกัด (มหาชน)
YCI,YCI,SET,สินค้าอุตสาหกรรม,ปิโตรเคมีและเคมีภัณฑ์,บริษัท ยงไทย จำกัด (มหาชน)
ZEN,ZEN,SET,เกษตรและอุตสาหกรรมอาหาร,อาหารและเครื่องดื่ม,บริษัท เซ็น คอร์ปอเรชั่น กรุ๊ป จำกัด (มหาชน)


# SET50 and SET100 sample for pilot


SET 50  https://www.settrade.com/C13_MarketSummary.jsp?detail=SET50

SET 100 https://www.settrade.com/C13_MarketSummary.jsp?detail=SET100


In [31]:
import lxml, requests
from lxml import html

src = "https://www.settrade.com/C13_MarketSummary.jsp?detail=SET50"
r = requests.get(src)
content = r.content.decode('tis-620')

_html = html.fromstring(content)
# _html.xpath("""//*[@id="maincontent"]/div/div[1]/div[1]/div/div[2]/div[3]/div/div/div/div/table/tbody/tr[15]/td[1]/a""")
links = _html.xpath("""//a[contains(@class,"link-stt") and contains(@href,'C13')]""") # link for each symbols

In [39]:
# links[0].getparent().getparent().tag  # 'tr'

symbols = []
for link in links:
    symbol = link.getparent().text_content().strip()
    symbols.append(symbol)

In [40]:
symbols

['SET',
 'SET50',
 'SET100',
 'sSET',
 'SETCLMV',
 'SETHD',
 'SETTHSI',
 'SETWB',
 'mai',
 'ADVANC',
 'AOT',
 'AWC',
 'BANPU',
 'BBL',
 'BDMS',
 'BEM',
 'BGRIM',
 'BH',
 'BJC\xa0<XD>',
 'BTS',
 'CBG',
 'CPALL\xa0<XD>',
 'CPF',
 'CPN',
 'CRC',
 'DELTA',
 'DTAC',
 'EA',
 'EGCO',
 'GLOBAL',
 'GPSC',
 'GULF',
 'HMPRO',
 'INTUCH',
 'IRPC',
 'IVL',
 'KBANK',
 'KTB',
 'KTC',
 'LH',
 'MINT',
 'MTC',
 'OSP',
 'PTT',
 'PTTEP',
 'PTTGC',
 'RATCH',
 'SAWAD',
 'SCB',
 'SCC',
 'TCAP',
 'TISCO',
 'TMB',
 'TOA',
 'TOP',
 'TRUE',
 'TU',
 'VGI',
 'WHA']

In [42]:
stocks = ['ADVANC',
 'AOT',
 'AWC',
 'BANPU',
 'BBL',
 'BDMS',
 'BEM',
 'BGRIM',
 'BH',]

In [43]:
stocks

['ADVANC', 'AOT', 'AWC', 'BANPU', 'BBL', 'BDMS', 'BEM', 'BGRIM', 'BH']

In [47]:
import pandas_datareader as pdr
import datetime as dt


stocks = ['ADVANC', 'AOT', 'AWC', 'BANPU', 'BBL', 'BDMS', 'BEM', 'BGRIM', 'BH']
start = dt.datetime(2014,1,1)

dataset = {}
for s in stocks:
    try:
        data = pdr.get_data_yahoo('%s.BK' % s,start=start)
        dataset[s] = data
    except Exception as e:
        print("Error: %s %s" % (s, str(e)))

In [48]:
dataset

{'ADVANC':              High    Low   Open  Close      Volume   Adj Close
 Date                                                          
 2014-01-02  198.5  187.0  198.0  187.0  12694200.0  137.609299
 2014-01-03  196.0  186.5  187.0  196.0  16987100.0  144.232224
 2014-01-06  203.0  191.0  195.0  201.0  10294700.0  147.911621
 2014-01-07  210.0  200.0  205.0  210.0  11479200.0  154.534515
 2014-01-08  210.0  203.0  208.0  206.0  13653100.0  151.591019
 ...           ...    ...    ...    ...         ...         ...
 2020-04-24  200.0  197.0  199.0  198.5   7015600.0  198.500000
 2020-04-27  201.0  197.5  199.0  200.0   5402400.0  200.000000
 2020-04-28  201.0  198.0  199.0  199.5   5255700.0  199.500000
 2020-04-29  201.0  199.0  199.5  201.0   3031100.0  201.000000
 2020-04-30  202.0  198.0  202.0  198.0   7327100.0  198.000000
 
 [1548 rows x 6 columns],
 'AOT':                  High    Low       Open  Close       Volume  Adj Close
 Date                                              